**Есть исторические данные(события взаимодействия пользователей с баннерами на сайте).
Формат данных :
date - дата когда было произведено действие.
banner_id - идентификатор баннера
adv_bid - ставка за 1 показ(в рублях)
placement - идентификатор страницы на которой произошло взаимодействие
user_id - id пользователя
action_type - тип события
Нужно сделать скрипт(подойдет любой вариант как и jupyter notebook, так и просто скрипт на python) который бы на вход принимал 2 параметра - ставку за 1 показ и  идентификатор страницы. А в результате выводил предсказание о том сколько показов получит этот баннер за  следующий день.**

In [266]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [284]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

In [299]:
start_data = pd.read_csv("test_data_3.csv")

In [300]:
start_data.groupby(['action_type']).count()

,date,banner_id,adv_bid,placement,user_id
action_type,,,,,
click,3614,3614,3614,3614,3614
favorite,58,58,58,58,58
impression,75314,75314,75314,75314,75314
to_cart,84,84,84,84,84
view,278713,278713,278713,278713,278713


В первую очередь понятно что совсем не нужны данные о пользователях.
Конечно, также, видно, что в данных есть повторы, но я не удаляю их потому что предположу, что один и тот же человек может несколько раз увидеть/нажать/добавить в козину один и тот же баннер

In [301]:
start_data.drop("user_id", axis=1,inplace=True)

In [302]:
start_data.head()

,date,banner_id,adv_bid,placement,action_type
0,2021-04-07,11089466256888358077,0.20,3,view
1,2021-04-07,12786233533693795083,1.20,2,view
2,2021-04-07,3597813970319619018,0.07,7,impression
3,2021-04-07,12786233533693795083,1.20,2,view
4,2021-04-07,12786233533693795083,1.20,2,view


Посмотрим о чем вообще датасет

In [297]:
print(len(start_data.banner_id.unique()),
len(start_data.adv_bid.unique()), len(start_data.date.unique()))

12 10 4


In [305]:
data = start_data.groupby(["banner_id", "date"])

In [ ]:
array = []
for banner_id, banner_df in data:
    print(banner_id)
    #print(banner_df)
    print("Количество показов:", len(banner_df), banner_df["adv_bid"].min(), banner_df["placement"].min())
    new_row = [banner_df["adv_bid"].min(), banner_df["placement"].min(), len(banner_df)]
    array.append(new_row)
    print(type(new_row))
processed_set = pd.DataFrame(array)

Убираю слишком маленькие значения из выборки


In [248]:
processed_set = processed_set[processed_set[2] > 66]

Заполняю тренировочный датасет данными (для одних и тех же ставок на одной и той же странице добавляю среднее)

In [259]:
arr = []
for set_id, set_df in processed_set.groupby([0,1]):
    print(set_df)
    print(set_df[2].mean())
    new_row = [set_df.iloc[0,0], set_df[1].iloc[0], set_df[2].mean()]
    arr.append(new_row)
processed_set_ = pd.DataFrame(arr)

        0  1     2
13  0.045  7  2885
2885.0
       0  1      2
6   0.07  7  12693
7   0.07  7  12679
8   0.07  7  12601
9   0.07  7  12730
10  0.07  7   6762
11  0.07  7   5284
12  0.07  7   5285
14  0.07  7   3549
8947.875
      0  1      2
2   0.2  3  20470
21  0.2  3  16697
23  0.2  3  25284
24  0.2  3   9264
25  0.2  3  10300
16403.0
      0   1    2
20  0.2  12  299
299.0
      0  1      2
0   0.3  2   8051
1   0.3  2  11763
15  0.3  2   2835
16  0.3  2  13553
17  0.3  2   6515
18  0.3  2   9028
19  0.3  2  10662
8915.285714285714
      0  1     2
22  0.3  3  6483
6483.0
        0  1      2
30  0.601  1  10219
31  0.601  1  10024
33  0.601  1   9824
34  0.601  1   9986
10013.25
     0   1      2
4  0.8  12   8711
5  0.8  12  11218
9964.5
       0  1      2
35  0.84  1  11489
36  0.84  1  11724
37  0.84  1  11677
38  0.84  1  11735
11656.25
      0  1      2
26  1.2  2  10196
27  1.2  2   6630
28  1.2  2   8609
8478.333333333334


In [304]:
processed_set_

,0,1,2
0,0.045,7,2885.000000
1,0.070,7,8947.875000
2,0.200,3,16403.000000
3,0.200,12,299.000000
4,0.300,2,8915.285714
5,0.300,3,6483.000000
6,0.601,1,10013.250000
7,0.800,12,9964.500000
8,0.840,1,11656.250000
9,1.200,2,8478.333333


In [261]:
X = processed_set_.drop(2, axis=1)
Y = processed_set_[2]

In [262]:
x_train, x_validation, y_train, y_validation = train_test_split(X, Y, random_state=13)

In [263]:
model = LinearRegression(normalize=True).fit(x_train, y_train)

In [264]:
model.score(x_validation, y_validation)

-0.2943373083296419

In [277]:
predictions = model.predict(x_train)

In [283]:
mae = mean_absolute_error(y_train, predictions)
print(y_train)
print(predictions)
print(mae)

1     8947.875000
4     8915.285714
7     9964.500000
8    11656.250000
9     8478.333333
0     2885.000000
2    16403.000000
Name: 2, dtype: float64
[ 8680.20139359 10281.77159962  7789.29436445 10967.78106727
 10948.01832795  8661.69454003  9921.48275472]
2746.5329771372667


Попробую постороить регерессию еще одним спобом

In [285]:
scaler = StandardScaler()
scaler.fit(processed_set_.drop(2, axis=1)) 
scaled_df = scaler.transform(processed_set_.drop(2, axis=1))

In [293]:
linear_regression_model = SGDRegressor(tol=.01, eta0=.1) 
linear_regression_model.fit(scaled_df, Y)
predictions = linear_regression_model.predict(scaled_df)
mae = mean_absolute_error(Y, predictions) 
print("RMAE: {}".format(np.sqrt(mae)))

RMAE: 54.230594656134286


In [294]:
print(y_train)
print(predictions)

1     8947.875000
4     8915.285714
7     9964.500000
8    11656.250000
9     8478.333333
0     2885.000000
2    16403.000000
Name: 2, dtype: float64
[ 6666.72076302  6711.16743557  8954.23992101  4427.35289762
  9635.01405826  9132.02661122 10673.13944282  5494.07303886
 11098.04963241 11235.09427011]


Мне не нравятся полученные результаты, лучше использовать первую модель